In [1]:
%cd ~

# clone the repo
!git clone https://github.com/EricJinhao/qtm350-final.git

# go into the repo
%cd qtm350-final

# confirm
import os
os.getcwd()


/Users/tianjinhao
fatal: destination path 'qtm350-final' already exists and is not an empty directory.
/Users/tianjinhao/qtm350-final


'/Users/tianjinhao/qtm350-final'

# 1. Create the SQLite DB and load the raw CSVs

In [2]:
import sqlite3
import pandas as pd
from pathlib import Path

# 1. make sure data folder exists (inside current repo)
db_path = Path("data/wdi.db")
db_path.parent.mkdir(parents=True, exist_ok=True)

# 2. connect to sqlite
conn = sqlite3.connect(db_path)

# 3. read your raw CSVs (already in data/raw/)
raw = pd.read_csv("data/raw/wdi_pophealth_data.csv")
meta = pd.read_csv("data/raw/wdi_pophealth_metadata.csv")

# 4. write them into sqlite as raw tables
raw.to_sql("raw_wdi_data", conn, if_exists="replace", index=False)
meta.to_sql("raw_wdi_metadata", conn, if_exists="replace", index=False)

print("Tables created in data/wdi.db")
conn.close()


Tables created in data/wdi.db


# Reshape & clean

In [5]:
import sqlite3
import pandas as pd
from pathlib import Path

# connect to db
conn = sqlite3.connect("data/wdi.db")

# load raw CSV from data/raw
raw_csv = pd.read_csv("data/raw/wdi_pophealth_data.csv")
print("Raw CSV shape:", raw_csv.shape)
print(raw_csv.head())

# write into sqlite
raw_csv.to_sql("raw_wdi_data", conn, if_exists="replace", index=False)

conn.commit()
conn.close()
print("Loaded raw_wdi_data into SQLite.")


Raw CSV shape: (14, 35)
  Country Name Country Code  \
0        China          CHN   
1        China          CHN   
2        China          CHN   
3        Japan          JPN   
4        Japan          JPN   

                                         Series Name     Series Code  \
0            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
1    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   
2  Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
3            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
4    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   

   1993 [YR1993]  1994 [YR1994]  1995 [YR1995]  1996 [YR1996]  1997 [YR1997]  \
0      69.492000      69.835000      70.363000      70.823000      71.227000   
1      50.800000      49.300000      47.500000      45.600000      43.600000   
2      13.469000       8.754000       8.652000       7.318000       6.427000   
3      79.303659      79.696829     

In [6]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/wdi.db")

print(pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn))

raw = pd.read_sql_query("SELECT * FROM raw_wdi_data LIMIT 5;", conn)
print(raw)

conn.close()


                  name
0     raw_wdi_metadata
1  wdi_pophealth_clean
2         raw_wdi_data
  Country Name Country Code  \
0        China          CHN   
1        China          CHN   
2        China          CHN   
3        Japan          JPN   
4        Japan          JPN   

                                         Series Name     Series Code  \
0            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
1    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   
2  Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
3            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
4    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   

   1993 [YR1993]  1994 [YR1994]  1995 [YR1995]  1996 [YR1996]  1997 [YR1997]  \
0      69.492000      69.835000      70.363000      70.823000      71.227000   
1      50.800000      49.300000      47.500000      45.600000      43.600000   
2      13.469000       8.754000       8.652000  

In [9]:
import sqlite3
import pandas as pd
from pathlib import Path

conn = sqlite3.connect("data/wdi.db")

# load from sqlite
raw = pd.read_sql_query("SELECT * FROM raw_wdi_data;", conn)

# 1. pick year columns: names start with 4-digit year, e.g. "1993 [YR1993]"
year_cols = [c for c in raw.columns if c[:4].isdigit()]
print("Year columns example:", year_cols[:5])

# 2. melt wide -> long
data_long = raw.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    value_vars=year_cols,
    var_name="year_raw",
    value_name="value"
)

# 3. clean column names
data_long = data_long.rename(columns={
    "Country Name": "country_name",
    "Country Code": "country_code",
    "Series Name": "indicator_name",
    "Series Code": "indicator_code",
})

# 4. extract numeric year from "1993 [YR1993]" -> 1993
data_long["year"] = data_long["year_raw"].str.slice(0, 4).astype(int)
data_long = data_long.drop(columns="year_raw")

print("Unique country codes:", data_long["country_code"].unique())
print("Unique indicator codes:", data_long["indicator_code"].unique())
print("Year range:", data_long["year"].min(), "to", data_long["year"].max())
print("Non-missing values count:", data_long["value"].notna().sum())

# 5. filter to our countries / indicators / years
keep_countries = ["CHN", "IND", "JPN"]
keep_indicators = ["SP.DYN.LE00.IN", "SH.DYN.MORT", "SP.ADO.TFRT"]

mask = (
    data_long["country_code"].isin(keep_countries)
    & data_long["indicator_code"].isin(keep_indicators)
    & data_long["year"].between(1993, 2023)
    & data_long["value"].notna()
)

clean = data_long.loc[mask].reset_index(drop=True)
print("Clean shape:", clean.shape)
print(clean.head())

# 6. save back to sqlite + csv
clean.to_sql("wdi_pophealth_clean", conn, if_exists="replace", index=False)
Path("data/clean").mkdir(parents=True, exist_ok=True)
clean.to_csv("data/clean/wdi_pophealth_clean.csv", index=False)

conn.close()
print("Recreated wdi_pophealth_clean table and CSV.")


Year columns example: ['1993 [YR1993]', '1994 [YR1994]', '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]']
Unique country codes: ['CHN' 'JPN' 'IND' None]
Unique indicator codes: ['SP.DYN.LE00.IN' 'SH.DYN.MORT' 'SP.ADO.TFRT' None]
Year range: 1993 to 2023
Non-missing values count: 279
Clean shape: (279, 6)
  country_name country_code  \
0        China          CHN   
1        China          CHN   
2        China          CHN   
3        Japan          JPN   
4        Japan          JPN   

                                      indicator_name  indicator_code  \
0            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
1    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   
2  Adolescent fertility rate (births per 1,000 wo...     SP.ADO.TFRT   
3            Life expectancy at birth, total (years)  SP.DYN.LE00.IN   
4    Mortality rate, under-5 (per 1,000 live births)     SH.DYN.MORT   

       value  year  
0  69.492000  1993  
1  50.800000  1993  
2  13.469

# ER Diagram

In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data/wdi.db")

# Countries table
conn.execute("DROP TABLE IF EXISTS countries;")
conn.execute("""
CREATE TABLE countries AS
SELECT DISTINCT
    country_code,
    country_name
FROM wdi_pophealth_clean;
""")

# Indicators table
conn.execute("DROP TABLE IF EXISTS indicators;")
conn.execute("""
CREATE TABLE indicators AS
SELECT DISTINCT
    indicator_code,
    indicator_name
FROM wdi_pophealth_clean;
""")

# Fact table
conn.execute("DROP TABLE IF EXISTS wdi_values;")
conn.execute("""
CREATE TABLE wdi_values AS
SELECT
    country_code,
    indicator_code,
    year,
    value
FROM wdi_pophealth_clean;
""")

conn.commit()
conn.close()
print("Created countries, indicators, and wdi_values tables.")


Created countries, indicators, and wdi_values tables.
